## Custom Sentiment Analysis 
these first few blocks are me copy-pasting the https://realpython.com/python-nltk-sentiment-analysis/#using-nltks-pre-trained-sentiment-analyzer 
code and learning how it works

In [25]:
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()
sia.polarity_scores("Wow, NLTK is really powerful!")

{'neg': 0.0, 'neu': 0.295, 'pos': 0.705, 'compound': 0.8012}

In [26]:
tweets = [t.replace("://", "//") for t in nltk.corpus.twitter_samples.strings()]

In [27]:
from random import shuffle

def is_positive(tweet: str) -> bool:
    """True if tweet has positive compound sentiment, False otherwise."""
    return sia.polarity_scores(tweet)["compound"] > 0

shuffle(tweets)
for tweet in tweets[:10]:
    print(">", is_positive(tweet), tweet)

> False @Clarrknulp I don't get your point. Are there Labour voters in Scotland who will be annoyed if Miliband doesn't do a deal with the SNP?
> True @josielong Don't panic.  My take: https//t.co/eeAxt0ZSLu.  Caroline Flint confirms it here: http//t.co/IG5OkrHtt1
> False @DeborahJaneOrr @DaAwob don't think many safe Labour seats left in my part of the woods. #snp
> True @RorkesDrift2 @mrmarksteel yet it's ok for Scotland to return 1 Tory MP, and get a Tory govt ?
> False RT @NickServini: .@welshlabour say tonight all of the comments from @Ed_Miliband in relation to the SNP apply to Plaid in exactly the same …
> False RT @WingsScotland: Be interesting to see what the papers report tomorrow. Miliband actually said nothing new, just worded something badly t…
> False RT @JeremyJHardy: The good thing about election time is the Tories work overtime to remind you they are horrible scum
> False @brittleyouth @Tom_J_Allen @AndrewFairbairn @batemanesque @Hegelbon @jameswheeler that was the wors

## my turn
Doing the same as above, but now with my corpus instead of tweets

In [28]:
#corpus as list of strings
import pandas as pd
df = pd.read_csv('C:\\Users\Connor\Desktop\\370\letterboxdCorpus\\all.csv')
reviews = df.reviews.tolist()
scoreless = []
for review in reviews:
    scoreless.append(review[5:])
print(scoreless[0:10])

['I highly recommend everyone get drunk with friends and watch this     ', "My introduction to John cena's absolutely delicious looking biceps     ", 'Not fast enough!     ', ' On march 24th 2023 Kanye West said he is no longer anti semitic because of this movie and Jonah Hill     ', "Why are old people having fun they're literally gonna die tomorrow 😭     ", 'Kids movie where dolls fight for their lives after the world ends in a terrifying way   somehow still less scary than happy feet     ', 'Yeah my ass was crying again my ass is always crying     ', 'Worst theater experience of my life fun movie tho     ', 'Wow wow wow wow wow WOW     ', ' Severe lack of solar systems but overall a good time     ']


In [29]:
shuffle(scoreless)
for review in scoreless[:10]:
    print(">", is_positive(review), review)

> True can't decide if that tech dude's mansion would be a good place to be quarantined or a terrifying / desolate / super drafty place to be quarantined. although at this point i'm starting to feel like *any* house — even the kind of desolate cliffside fortress that 's basically just what a billionaire tech bro might design after watching HEAT once — would be nice to be stuck inside these days. at least my son can't crawl yet. and my...     
> True  really quietly affecting work from my dear friend jacob. this one is introspective and soft, and definitely his most mature short film yet. a muted look at the way people drift in and out of our lives, and how sometimes they're called back to our minds even if we haven't thought about them for a while. and i was lucky to be an advising editor and help out with the poster as well, it brings me immense joy and pride to see jacob's work come to life every time. you can watch the short film here     
> True I am so not the target audience for 

I honestly don't mind the supposed ~60% accuracy. I do like the results.

## generating top 100s

In [30]:
unwanted = nltk.corpus.stopwords.words("english")
unwanted.extend([w.lower() for w in nltk.corpus.names.words()])
#
#here's my own injection, which makes a list of words in positive reviews and a list
#of negative reviews, so that I don't need to use the nltk.corpus movie reviews
pos_category = []
neg_category = []
for review in scoreless:
    words = review.split()
    if is_positive(review): 
        for word in words:
            pos_category.append(word)
    else:
        for word in words:
            neg_category.append(word)


def skip_unwanted(pos_tuple):
    word, tag = pos_tuple
    if not word.isalpha() or word in unwanted:
        return False
    if tag.startswith("NN"):
        return False
    return True

#here's where it trains off my stuff rather than nltk movie reviews
positive_words = [word for word, tag in filter(
    skip_unwanted,
    nltk.pos_tag(pos_category)
)]
negative_words = [word for word, tag in filter(
    skip_unwanted,
    nltk.pos_tag(neg_category)
)]

In [31]:
positive_fd = nltk.FreqDist(positive_words)
negative_fd = nltk.FreqDist(negative_words)

common_set = set(positive_fd).intersection(negative_fd)

for word in common_set:
    del positive_fd[word]
    del negative_fd[word]

top_100_positive = {word for word, count in positive_fd.most_common(100)}
top_100_negative = {word for word, count in negative_fd.most_common(100)}
print("POSTIVE:")
print(top_100_positive)
print("NEGATIVE:")
print(top_100_negative)

POSTIVE:
{'satisfied', 'owning', 'prominent', 'thankful', 'kissing', 'joyful', 'easiest', 'inspirational', 'subscribing', 'cutest', 'Subscribing', 'beguiling', 'delighted', 'breathlessly', 'hopeful', 'Liked', 'owed', 'costuming', 'intelligent', 'thrilled', 'stultifying', 'approach', 'mined', 'curly', 'superb', 'sharply', 'lovelorn', 'obscure', 'pls', 'ground', 'inclusive', 'hong', 'saturday', 'riotous', 'subverts', 'pleasantly', 'remained', 'richly', 'celebrate', 'impoverished', 'blessed', 'harmless', 'minding', 'emerges', 'improvisational', 'entered', 'united', 'documented', 'WANT', 'rebellious', 'kidding', 'uncharted', 'smile', 'Essentially', 'extend', 'hagiographic', 'perceive', 'eps', 'gain', 'transcends', 'grateful', 'cringe', 'breakout', 'panoramic', 'libertarian', 'vibe', 'smooth', 'luckily', 'mandatory', 'seamless', 'Preparing', 'table', 'underwhelming', 'enthusiastic', 'captivated', 'sporadically', 'insistently', 'subconsciously', 'award', 'sugary', 'contemplative', 'intricate

So, there are the rough top 100 positive and negative words in my corpus. Apart from a few questionable results here and there, I'd consider it mostly good.

Here's an idea. I'm going to redefine my pos/neg_category variables using the user scores attached to reviews. I'm hoping it gives me more accurate evaluations than the polarity scores

In [32]:
import nltk
from nltk.tokenize import word_tokenize

#8s, 9s, and 10/10s
good_movies = []
#3s, 2s, and 1/10s
bad_movies = []
#var for later
sentences = []
for review in reviews:
    if review.startswith("10") or review.startswith("9") or review.startswith("8"):
        lower = review[5:].lower()
        tokens = word_tokenize(lower)
        for token in tokens:
            good_movies.append(token)
    if review.startswith("1/") or review.startswith("2") or review.startswith("3"):
        lower = review[5:].lower()
        tokens = word_tokenize(lower)
        for token in tokens:
            bad_movies.append(token)
    lower2 = review[5:].lower()
    tokens2 = word_tokenize(lower2)
    sentences.append(tokens2)
print(bad_movies[0:50])

['i', 'highly', 'recommend', 'everyone', 'get', 'drunk', 'with', 'friends', 'and', 'watch', 'this', 'my', 'introduction', 'to', 'john', 'cena', "'s", 'absolutely', 'delicious', 'looking', 'biceps', 'i', 'like', 'that', 'one', 'robert', 'de', 'niro', 'comedy', 'where', 'he', 'shows', 'his', 'cock', 'and', 'fucks', 'like', 'crazy', 'this', 'is', 'not', 'that', 'one', 'tho', 'i', 'know', 'a', 'guy', 'named', 'aksel']


I'm too lazy to go up and make a function

In [33]:
#now processing new stuff see line 4 and 8
positive_words = [word for word, tag in filter(
    skip_unwanted,
    nltk.pos_tag(good_movies)
)]
negative_words = [word for word, tag in filter(
    skip_unwanted,
    nltk.pos_tag(bad_movies)
)]
positive_fd = nltk.FreqDist(positive_words)
negative_fd = nltk.FreqDist(negative_words)

common_set = set(positive_fd).intersection(negative_fd)

for word in common_set:
    del positive_fd[word]
    del negative_fd[word]

top_100_positive = {word for word, count in positive_fd.most_common(100)}
top_100_negative = {word for word, count in negative_fd.most_common(100)}
print("POSTIVE:")
print(top_100_positive)
print("NEGATIVE:")
print(top_100_negative)

POSTIVE:
{'allows', 'bodily', 'subscribing', 'gothic', 'inherent', 'grounded', 'vivid', 'detailed', 'stronger', 'scariest', 'warm', 'devastating', 'empathetic', 'tiny', 'noir', 'moves', 'masterful', 'vital', 'sudden', 'ep', 'fantastical', 'adds', 'trapped', 'rewatch', 'pulls', 'grew', 'shaking', 'gleefully', 'tense', 'internal', 'sees', 'wrapped', 'neon', 'delicate', 'exquisite', 'twin', 'sensual', 'familial', 'experimental', 'atmosphere', 'marvelous', 'loving', 'elegant', 'gorgeously', 'builds', 'detective', 'apocalyptic', 'melancholic', 'lying', 'expertly', 'harsh', 'layered', 'harrowing', 'claustrophobic', 'flawed', 'grateful', 'ghostly', 'phenomenal', 'indelible', 'fitting', 'finds', 'hysterical', 'showcasing', 'unexpected', 'surreal', 'uses', 'magnificent', 'underrated', 'breaking', 'nightmarish', 'shown', 'similar', 'touching', 'essential', 'dreamy', 'vividly', 'classical', 'gentle', 'allow', 'adding', 'fractured', 'vicious', 'beautifully', 'flows', 'existential', 'impeccable', '

Even cooler results. Although maybe affected by a smaller sample size, since I'm seeing the infrequently used "fuckahhhhhhhhh" in the negative.

So my best idea to find more stuff with word embeddings is to process my positive and negative words and see what pops up frequently in .most_similar calls

In [34]:
from gensim.models import Word2Vec
print(len(sentences))
model = Word2Vec(sentences = sentences,
                 window = 2, 
                 min_count =1,
                 vector_size = 5)

25344


In [35]:
#similar_words = model.wv.most_similar('sensual', topn= 10)
# ^ list of tuples. first word = [0][0]
#making a dictionary in the (word, count) format
near_positive = {}
for word in top_100_positive:
    sim = model.wv.most_similar(word, topn= 500)
    for i in range(500):
        synonym = sim[i][0]
        #print(word)
        if synonym in near_positive:
            near_positive[synonym] += 1
        else:
            near_positive[synonym] = 1
sorted_pos = {k: v for k, v in sorted(near_positive.items(), key=lambda item: item[1], reverse=True)}
print("WORDS \"SIMILAR TO\" POSITIVE WORDS:")
message = ""
for key, value in list(sorted_pos.items())[:200]:
    message += f"{key}:{value}  "
print(message)

WORDS "SIMILAR TO" POSITIVE WORDS:
possessed:24  mania:24  ludicrous:24  intense:24  lego:24  gimmick:23  unexpectedly:23  comedies:23  tj:23  opens:23  setting:23  convoluted:23  deadly:23  vigilante:22  artificial:22  sustained:22  tissue:22  rhythms:22  shark:22  thrills:22  soft:22  chase:22  lush:22  junk:22  casual:22  absurdly:22  overlong:22  quiet:22  fits:22  fat:22  questioning:21  lil:21  madman:21  misery:21  variety:21  fractured:21  develops:21  grisly:21  coda:21  awkwardness:21  stretched:21  delightfully:21  hypnotic:21  messy:21  60s:21  aesthetic:21  gag:21  sincere:21  subverts:21  unhinged:21  row:21  apocalypse:21  holiday:21  melancholy:21  source:21  macho:21  tragic:21  clean:21  suggesting:21  rendered:21  aces:21  bland:21  tantalizing:21  jagged:21  pov:20  crossfire:20  stunt:20  lengthy:20  surfaces:20  moody:20  drug:20  pop:20  locations:20  troop:20  drenched:20  absent:20  underneath:20  bloody:20  analog:20  hayley:20  occasional:20  grieving:20  tre

In [36]:
near_negative = {}
for word in top_100_negative:
    sim = model.wv.most_similar(word, topn= 500)
    for i in range(500):
        synonym = sim[i][0]
        #print(word)
        if synonym in near_negative:
            near_negative[synonym] += 1
        else:
            near_negative[synonym] = 1
sorted_neg = {k: v for k, v in sorted(near_negative.items(), key=lambda item: item[1], reverse=True)}
print("WORDS \"SIMILAR TO\" NEGATIVE WORDS:")
message = ""
for key, value in list(sorted_neg.items())[:200]:
    message += f"{key}:{value}  "
print(message)

WORDS "SIMILAR TO" NEGATIVE WORDS:
forbidden:9  2022-2023:9  plotline:9  narcissists:9  flick:8  amounts:8  sporadically:8  deserved:8  embodiment:8  regrettably:8  hillary:8  accurately:8  moods:8  dunhills:8  bubble-blowing:8  wrote:8  glue:8  panic:8  scares:8  places:8  whodunnit:8  cave:8  knight:8  semi-realism:8  woven:8  tune:8  shorts:8  factor:8  sov:7  jesse:7  kingdom:7  showcasing:7  demonic:7  misogynist:7  anthony:7  creatures:7  pool:7  skills:7  quote:7  attendant:7  parker:7  kidnapping:7  lowkey:7  billed:7  keaton:7  socialist:7  disappointingly:7  evidently:7  compares:7  slate:7  allows:7  methods:7  bahston:7  puns:7  investigate:7  bizarrely:7  mud-and-guts:7  aghdashloo:7  screenings:7  abandon:7  dump:7  reminded:7  ozzy:7  painstakingly:7  bizarro:7  charmed:7  damned:7  asset:7  adulthood:7  education:7  ugliest-looking:7  dirtbikes:7  bought:7  spectre:7  dig:7  treats:7  cinematographic:7  duller:7  language:7  00100000:7  no-budget:7  over-cut:7  takakura

These results kinda suck, but improve some as i turn up the topn parameter. I imagine as the count values decrease, the less reliable the words get.